https://www.kaggle.com/laknath/blend-it-all

In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
import xgboost as xgb
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
# cleaned text files. Refer notebook "Preprocessing Text"
train_file = 'train_cleaned.csv' #train.csv
test_file = 'test_cleaned.csv' #test.csv

train = pd.read_csv('../input/' + train_file).fillna(' ')#.sample(1000)
test = pd.read_csv('../input/' + test_file).fillna(' ')#.sample(1000)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

train = train.loc[:,class_names]

### Finding corelation

In [ ]:
fname = '../submissions/ensemble/submissions.txt'

with open(fname) as f:
    for line in f:
        print(line)

In [13]:
#! ls -la '../submissions/ensemble'
fold1 = '../submissions/ensemble/'

submissions = [
    'blend_it_all.gru_glove.fasttext.9868.csv',
    'cnn_glove_128_95p.10fold_weighted_avg.9850.csv',
    'blend_it_all.9867.csv',
    'dpcnn_glove_32_95p_reg.20fold_avg.9835.csv',
    'gru_glove_128_95p_reg.10fold_avg.9853.csv',    
    'gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csv',
    'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv',    
    'lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv',
    'lstm_fasttext_256_95p_reg.5fold_avg.9841.csv',
    'cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_5_fasttext_maxpool_ep2_batch_256_9837.csv',
    'cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_2_fasttext_maxpool_ep2_batch_128_9834.csv',
    'lstm_l1_256_spatial_dr_0_4_gpu_dr_0_5_fasttext_maxpool_ep2_batch_128_nadam_9834.csv',
    'cnn_2_window_2_3_filter_64_l1_lstm80_spatial_dr_0_4_lstm_dr_0_2_fasttext_ep3_batch_128_9833.csv',
    'gru_l1_gru128_spatial_dr_0_4_gpu_dr_0_5_fasttext_maxpool_ep1_batch_128_9832.csv',
    'lstm_64_biLSTM_timedistributed_fasttext_pooling_batch32.990567.9831.csv',
    
    'cnn_2_window_2_3_filter_64_l1_lstm80_spatial_dr_0_4_lstm_dr_0_2_dense_64_dr_0_2_glove_ep3_batch_128_9830.csv',
    'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_fasttext_ep4_batch_256_nadam_001_9833.csv',
    'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_glove_ep5_batch_128_nadam_001_9827.csv',
    'toxic_lr_char_tdidf.9792.csv',
    
    'blend_of_only_blends.csv'
]

len(submissions)

20

In [11]:
#! ls -la '../submissions/ensemble'
fold2 = '../submissions/ensemble/downloads/'

submissions = [
    'bi-gru-cnn.9850.csv', 
    'bi-lstm-cnn.9841.csv',    
    'pooled-gru-fasttext.9833.csv',     
    'pooled_gru_glove_r.9815.csv',  
    'pooled-gru-with-preprocessing.csv',
    'hight_of_blending.9835.csv', 
    'lvl0_wordbatch_clean_oof.9805.csv', 
    'superblend_1.csv', 
    'lvl0_lgbm_clean_oof.9792.csv', 
    'lvl0_wordbatch_clean_sub.9805.csv', 
    'tidy_xgb_glm.9784.csv', 
    'corr_blend.9855.csv', 
    'lvl0_lgbm_clean_sub.9792.csv', 
    'oof-stacking-regime.9858.csv',
    'toxic-avenger.9823.csv'
]    

len(submissions)

15

In [10]:
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/50827

import pandas as pd
import sys
from scipy.stats import ks_2samp

#first_file = fold1+"gru_glove_128_95p_reg.10fold_avg.9853.csv"
#second_file = fold2+"pooled-gru-with-preprocessing.9823.csv"

first_file = fold1+"blend_of_only_blends.csv" #bi-lstm-cnn.9841.csv
second_file = fold2+"bi-gru-cnn.9850.csv"
print(first_file, second_file)


def corr(first_file, second_file):
    # assuming first column is `class_name_id`
    first_df = pd.read_csv(first_file, index_col=0)
    second_df = pd.read_csv(second_file, index_col=0)
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    for class_name in class_names:
        # all correlations
        print('\n Class: %s' % class_name)
        print(' Pearson\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='pearson'))
        print(' Kendall\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='kendall'))
        print(' Spearman\'s correlation score: %0.6f' %
              first_df[class_name].corr(
                  second_df[class_name], method='spearman'))
        ks_stat, p_value = ks_2samp(first_df[class_name].values,
                                    second_df[class_name].values)
        print(' Kolmogorov-Smirnov test:    KS-stat = %.6f    p-value = %.3e\n'
              % (ks_stat, p_value))

corr(first_file, second_file)

# pooled-gru-fasttext.9833.csv')
# BL version - lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv + 
#fast_bl = pd.read_csv(fold1+'lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv'

#gru_glove_128_95p_reg.10fold_avg.9853.csv
#lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv

#gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csv
#lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv


../submissions/ensemble/blend_of_only_blends.csv ../submissions/ensemble/downloads/bi-gru-cnn.9850.csv

 Class: toxic
 Pearson's correlation score: 0.976440
 Kendall's correlation score: 0.796785
 Spearman's correlation score: 0.937776
 Kolmogorov-Smirnov test:    KS-stat = 0.651374    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.919514
 Kendall's correlation score: 0.659038
 Spearman's correlation score: 0.837724
 Kolmogorov-Smirnov test:    KS-stat = 0.929187    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.978433
 Kendall's correlation score: 0.720504
 Spearman's correlation score: 0.880524
 Kolmogorov-Smirnov test:    KS-stat = 0.784395    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.884541
 Kendall's correlation score: 0.548316
 Spearman's correlation score: 0.746852
 Kolmogorov-Smirnov test:    KS-stat = 0.988953    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.967026
 Kendall's c

In [ ]:
# compare all against all
for f1 in submissions:
    for f2 in submissions:
        if f1 != f2:
            print("Corelation between : ", f1, f2)
            corr(fold1+f1, fold2+f2)

In [14]:
# compare one against all

#first_file = "blend_it_all.9867.csv"
first_file = "blend_of_only_blends.csv" #bi-lstm-cnn.9841.csv
#second_file = fold2+"bi-gru-cnn.9850.csv"

for f in submissions:
    if f != first_file:
        print("Corelation between : ", f, first_file)
        corr(fold1+f, fold1+first_file)

Corelation between :  blend_it_all.gru_glove.fasttext.9868.csv blend_of_only_blends.csv

 Class: toxic
 Pearson's correlation score: 0.996879
 Kendall's correlation score: 0.956280
 Spearman's correlation score: 0.996685
 Kolmogorov-Smirnov test:    KS-stat = 0.570989    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.993470
 Kendall's correlation score: 0.953987
 Spearman's correlation score: 0.993661
 Kolmogorov-Smirnov test:    KS-stat = 0.865908    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.997051
 Kendall's correlation score: 0.956892
 Spearman's correlation score: 0.996093
 Kolmogorov-Smirnov test:    KS-stat = 0.697390    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.989746
 Kendall's correlation score: 0.935049
 Spearman's correlation score: 0.990526
 Kolmogorov-Smirnov test:    KS-stat = 0.964783    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.995665
 Kendall's correlation scor


 Class: toxic
 Pearson's correlation score: 0.958367
 Kendall's correlation score: 0.791949
 Spearman's correlation score: 0.935077
 Kolmogorov-Smirnov test:    KS-stat = 0.610274    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.942101
 Kendall's correlation score: 0.632107
 Spearman's correlation score: 0.810485
 Kolmogorov-Smirnov test:    KS-stat = 0.898024    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.973974
 Kendall's correlation score: 0.726162
 Spearman's correlation score: 0.884841
 Kolmogorov-Smirnov test:    KS-stat = 0.746487    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.901081
 Kendall's correlation score: 0.557056
 Spearman's correlation score: 0.749271
 Kolmogorov-Smirnov test:    KS-stat = 0.971873    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.968072
 Kendall's correlation score: 0.742971
 Spearman's correlation score: 0.900610
 Kolmogorov-Smirnov test:    KS-stat


 Class: toxic
 Pearson's correlation score: 0.972791
 Kendall's correlation score: 0.762934
 Spearman's correlation score: 0.914687
 Kolmogorov-Smirnov test:    KS-stat = 0.656956    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.941097
 Kendall's correlation score: 0.608908
 Spearman's correlation score: 0.788356
 Kolmogorov-Smirnov test:    KS-stat = 0.901002    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.975539
 Kendall's correlation score: 0.714852
 Spearman's correlation score: 0.875483
 Kolmogorov-Smirnov test:    KS-stat = 0.762026    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.868688
 Kendall's correlation score: 0.532364
 Spearman's correlation score: 0.724856
 Kolmogorov-Smirnov test:    KS-stat = 0.981066    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.965705
 Kendall's correlation score: 0.738912
 Spearman's correlation score: 0.898078
 Kolmogorov-Smirnov test:    KS-stat


 Class: toxic
 Pearson's correlation score: 0.971976
 Kendall's correlation score: 0.832279
 Spearman's correlation score: 0.958787
 Kolmogorov-Smirnov test:    KS-stat = 0.623456    p-value = 0.000e+00


 Class: severe_toxic
 Pearson's correlation score: 0.893972
 Kendall's correlation score: 0.748989
 Spearman's correlation score: 0.911060
 Kolmogorov-Smirnov test:    KS-stat = 0.942271    p-value = 0.000e+00


 Class: obscene
 Pearson's correlation score: 0.966665
 Kendall's correlation score: 0.811579
 Spearman's correlation score: 0.947852
 Kolmogorov-Smirnov test:    KS-stat = 0.773400    p-value = 0.000e+00


 Class: threat
 Pearson's correlation score: 0.868873
 Kendall's correlation score: 0.713301
 Spearman's correlation score: 0.888326
 Kolmogorov-Smirnov test:    KS-stat = 0.990259    p-value = 0.000e+00


 Class: insult
 Pearson's correlation score: 0.952055
 Kendall's correlation score: 0.810600
 Spearman's correlation score: 0.949004
 Kolmogorov-Smirnov test:    KS-stat

In [ ]:
blend_it_all.9867.csv
-------

gru_glove_128_95p_reg.10fold_avg.9853.csv
gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csvf
lstm_l1_256_spatial_dr_0_4_gpu_dr_0_5_fasttext_maxpool_ep2_batch_128_nadam_9834
lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv
lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv
cnn_2_window_2_3_l1_lstm256_spatial_dr_0_4_lstm_dr_0_5_fasttext_maxpool_ep2_batch_256_9837.csv
toxic_lr_char_tdidf.9792.csv - 1


In [ ]:
# https://www.kaggle.com/peterhurford/lgb-gru-lr-lstm-nb-svm-average-ensemble/code

import numpy as np
import pandas as pd


gru = pd.read_csv('../input/pooled-gru-fasttext/submission.csv') # PL score 0.9829
lstm_nb_svm = pd.read_csv('../input/minimal-lstm-nb-svm-baseline-ensemble/submission.csv') # 0.9811
lr = pd.read_csv('../input/logistic-regression-with-words-and-char-n-grams/submission.csv') # 0.9788
lgb = pd.read_csv('../input/lightgbm-with-select-k-best-on-tfidf/lgb_submission.csv') # 0.9785


# The value of an ensemble is (a) the individual scores of the models and
# (b) their correlation with one another. We want multiple individually high
# scoring models that all have low correlations. Based on this analysis, it
# looks like these kernels have relatively low correlations and will blend to a
# much higher score.
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
for label in labels:
    print(label)
    print(np.corrcoef([lgb[label].rank(pct=True), gru[label].rank(pct=True), lr[label].rank(pct=True), lstm_nb_svm[label].rank(pct=True)]))

submission = pd.DataFrame()
submission['id'] = lgb['id']
for label in labels:
    submission[label] = lgb[label].rank(pct=True) * 0.15 + gru[label].rank(pct=True) * 0.4 + lr[label].rank(pct=True) * 0.15 + lstm_nb_svm[label].rank(pct=True) * 0.3

submission.to_csv('submission.csv', index=False)

### Blending

In [4]:
fold1 = '../submissions/ensemble/'
fold2 = '../submissions/ensemble/downloads/'

# https://www.kaggle.com/codename007/toxic-avenger-spin-lb-0-9832
ave = pd.read_csv(fold2+"toxic-avenger.9823.csv")

# https://www.kaggle.com/tunguz/blend-of-blends-1/output 
supbl= pd.read_csv(fold2+'superblend_1.csv')

# https://www.kaggle.com/nuhsikander/blend-of-blends-can-be-toxic-lb-0-9837 0.9837
best = pd.read_csv(fold2+'hight_of_blending.9835.csv')

# https://www.kaggle.com/hhstrand/oof-stacking-regime 0.9858
oofs = pd.read_csv(fold2+'oof-stacking-regime.9858.csv')

# https://www.kaggle.com/antmarakis/another-blend-tinkered-by-correlation - 0.9855
corrbl = pd.read_csv(fold2+'corr_blend.9855.csv')

# https://www.kaggle.com/ogrellier/wordbatch-fm-ftrl-using-mse-lb-0-9804 0.9805 FM FLTR
wordbtch = pd.read_csv(fold2+'lvl0_wordbatch_clean_sub.9805.csv')

# GRU pooling with R - https://www.kaggle.com/krosam/why-a-such-low-score-with-r-and-keras 0.9835
rkera = pd.read_csv(fold2+'pooled_gru_glove_r.9815.csv')

# https://www.kaggle.com/ogrellier/lgbm-with-words-and-chars-n-gram 0.9792
lgbm = pd.read_csv(fold2+'lvl0_lgbm_clean_sub.9792.csv')

# https://www.kaggle.com/kailex/tidy-xgboost-glmnet-text2vec-lsa 0.9786
tidy = pd.read_csv(fold2+'tidy_xgb_glm.9784.csv')

# https://www.kaggle.com/eashish/bidirectional-gru-with-convolution 0.9850
grucnn = pd.read_csv(fold2+'bi-gru-cnn.9850.csv')
grucnn_bl = pd.read_csv(fold1+'cnn_glove_128_95p.10fold_weighted_avg.9850.csv')

# https://www.kaggle.com/eashish/bidirectional-gru-with-convolution
bilst = pd.read_csv(fold2+'bi-lstm-cnn.9841.csv')

# https://www.kaggle.com/prashantkikani/pooled-gru-glove-with-preprocessing 0.9823
gruglo = pd.read_csv(fold2+"pooled-gru-with-preprocessing.9823.csv")
# BL version
gruglo_bl = pd.read_csv(fold1+"gru_glove_128_95p_reg.10fold_avg.9853.csv")

# https://www.kaggle.com/zhbain/pooled-gru-fasttext-6c07c9/code #0.9833
fast = pd.read_csv(fold2+'pooled-gru-fasttext.9833.csv')
# BL version - lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv + 
fast_bl = pd.read_csv(fold1+'lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv')
fast_bl2 = pd.read_csv(fold1+'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv')

dpcnn = pd.read_csv(fold1+'dpcnn_glove_32_95p_reg.20fold_avg.9835.csv')

char_tdidf = pd.read_csv(fold1+'toxic_lr_char_tdidf.9792.csv')


#gru_glove_128_95p_reg.10fold_avg.9853.csv
#lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv

#gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csv
#lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv
# -------
#gru_glove_128_95p_reg.10fold_avg.9853.csv
#lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv
#lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv
#gru_fasttext_128_95p_reg.5fold_weighted_avg.9839.csv
#toxic_lr_char_tdidf.9792.csv - 1


In [ ]:
# 0.9867

b1 = best.copy()
col = best.columns

col = col.tolist()
col.remove('id')

for i in col:
    b1[i] = (fast[i] * 2 + gruglo[i] * 2 + grucnn[i] * 4 + ave[i] + supbl[i] * 2 + best[i] * 4 
             +  wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4) /  33
    
b1.to_csv('../submissions/blend_it_all.gru_glove.fasttext.dpcnn.cnn.csv', index = False)

In [ ]:
# 0.9868

b1 = best.copy()
col = best.columns

col = col.tolist()
col.remove('id')

for i in col:
    b1[i] = (fast[i] * 2 + gruglo[i] * 2 + ((grucnn[i] + grucnn_bl[i])/2) * 4 + ave[i] + supbl[i] * 2 + best[i] * 4 
             +  wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4
             + gruglo_bl[i] * 4 + fast_bl[i] * 2 + dpcnn[i] * 3 + char_tdidf) /  43
    
b1.to_csv('../submissions/blend_it_all.gru_glove.fasttext.dpcnn.cnn.csv', index = False)

In [ ]:
# expreiment

b1 = best.copy()
col = best.columns

col = col.tolist()
col.remove('id')

for i in col:
    b1[i] = (ave[i] + supbl[i] * 2 + best[i] * 4 + oofs[i] * 5 + corrbl[i] * 4
             + wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i]
             + gruglo[i] * 2 + ((grucnn[i] + grucnn_bl[i])/2) * 4 +  fast[i] * 2 +
              bilst[i] * 4 + gruglo_bl[i] * 4 + fast_bl[i] * 2 + dpcnn[i] * 3 + char_tdidf) /  43
    
b1.to_csv('../submissions/blend_it_all.gru_glove.fasttext.dpcnn.cnn.csv', index = False)

## Records

In [6]:
# mixure of all blends and external models that won't be tried

b1 = best.copy()
col = best.columns

col = col.tolist()
col.remove('id')

for i in col:
    b1[i] = (ave[i] + supbl[i] * 2 + best[i] * 4 + oofs[i] * 5 + corrbl[i] * 4 #16
             + wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i]) /  21
    
#b1.to_csv('../submissions/blend_of_only_blends.csv', index = False)

In [ ]:
#### v3

gruglo = pd.read_csv(fold2+"pooled-gru-with-preprocessing.9823.csv")
gruglo_bl = pd.read_csv(fold1+"gru_glove_128_95p_reg.10fold_avg.9853.csv")

fast = pd.read_csv(fold2+'pooled-gru-fasttext.9833.csv')
fast_bl = pd.read_csv(fold1+'lstm_fasttext_256_95p_reg.5fold_weighted_avg.9842.csv')
fast_bl2 = pd.read_csv(fold1+'lstm_l1_64_spatial_dr_0_4_lstm_dr_0_2_timedistributed_dense_relu_100_dr_0_1_amaxpool_dense_64_dr_0_1_glove_ep6_batch_256_nadam_001_9843.csv')

dpcnn = pd.read_csv(fold1+'dpcnn_glove_32_95p_reg.20fold_avg.9835.csv')

char_tdidf = pd.read_csv(fold1+'toxic_lr_char_tdidf.9792.csv')


for i in col:
    b1[i] = (fast[i] + ((grucnn[i] + grucnn_bl[i])/2) * 4 + ave[i] + supbl[i] * 2 + best[i] * 4 # 12
             +  wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4 #18
             + gruglo_bl[i] * 4 + ((fast_bl[i] + fast_bl2[i])/2) * 2 + dpcnn[i] + char_tdidf[i]) / 38 #8
    
#b1.to_csv('../submissions/blend_it_all.gru_glove.fasttextavg.dpcnn.cnn.csv', index = False)

In [ ]:
### v2

for i in col:
    b1[i] = (fast[i] * 2 + gruglo[i] * 2 + ((grucnn[i] + grucnn_bl[i])/2) * 4 + ave[i] + supbl[i] * 2 + best[i] * 4 
             +  wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4
             + gruglo_bl[i] * 4 + fast_bl[i] * 2 + dpcnn[i] * 3 + char_tdidf) /  43
    
#b1.to_csv('../submissions/blend_it_all.gru_glove.fasttext.dpcnn.cnn.csv', index = False)

In [ ]:
#### v1

for i in col:
    b1[i] = (fast[i] * 2 + grucnn[i] * 4 + ave[i] + supbl[i] * 2 + best[i] * 4 
             +  wordbtch[i] * 2 + lgbm[i] * 2 + tidy[i] + bilst[i] * 4 + oofs[i] * 5 + corrbl[i] * 4
             + gruglo_bl[i] * 4 + fast_bl[i] * 2 + dpcnn[i] * 3) /  43
    
#b1.to_csv('../submissions/blend_it_all.9853.9843.csv', index = False)

### Components of blends

* ave - https://www.kaggle.com/codename007/toxic-avenger-spin-lb-0-9823
    - https://www.kaggle.com/demesgal/lstm-with-bn-nb-svm-lr-on-conv-ai-lb-0-041

* supbl - https://www.kaggle.com/tunguz/blend-of-blends-1/output 
    - lgb-gru-lr-lstm-nb-svm-average-ensemble/submission.csv
    - toxic-one-more-b8bce2/one_more_blend.csv
    
* best - https://www.kaggle.com/nuhsikander/blend-of-blends-can-be-toxic-lb-0-9837 0.9837
    - glove - /input/toxic-glove/glove.csv
    - /input/fasttext-like-baseline-with-keras-lb-0-053/submission_bn_fasttext.csv
    - /input/toxicfile/toxicave.csv'
    - lstm /input/toxicfiles/baselinelstm0069.csv
    - svm = /input/toxic-nbsvm/nbsvm.csv
    - /input/logistic-regression-with-words-and-char-n-grams/submission.csv
    - input/toxic-simple-blending/blend_sub.csv
    - input/lstm-with-bn-nb-svm-lr-on-conv-ai-lb-0-041/submission_ensemble.csv
    - input/toxicfile/sub9821.csv

* oofs - https://www.kaggle.com/hhstrand/oof-stacking-regime 0.9858
    - 29: LightGBM trained on Fasttext (CV: 0.9765, LB: 0.9620)
    - 51: Logistic regression with word and char n-grams (CV: 0.9858, LB: ?)
    - 52: LSTM trained on Fasttext (CV: ?, LB: 0.9851)

* corrbl - https://www.kaggle.com/antmarakis/another-blend-tinkered-by-correlation - 0.9855
    - lstm = pd.read_csv('../input/bidirectional-lstm-with-convolution/submission.csv') #0.9841
    - lr = pd.read_csv('../input/tuned-logreg-oof-files/submission-tuned-LR-01.csv') #0.9800
    - gru_cnn = pd.read_csv('../input/bi-gru-cnn-poolings/submission.csv') #0.9841
    - r_script = pd.read_csv('../input/why-a-such-low-score-with-r-and-keras/submission.csv') #0.9824

* https://www.kaggle.com/tilii7/ensemble-weights-minimization-vs-mcmc
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/51546
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/50913
* https://www.kaggle.com/peterhurford/lgb-gru-lr-lstm-nb-svm-average-ensemble/output
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/50827
* https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/51274
* https://www.kaggle.com/reppic/lazy-ensembling-algorithm
* https://www.kaggle.com/hhstrand/oof-stacking-regime/code